In [ ]:
#database demo là dữ liệu nguồn
#database demo1 là data warehouse

In [10]:
import psycopg2

# Kết nối tới cơ sở dữ liệu 'demo'
conn_demo = psycopg2.connect(
    dbname="demo",
    user="postgres",
    password="password",
    host="localhost",
    port="5432"
)

cursor_demo = conn_demo.cursor()

# Kết nối tới cơ sở dữ liệu 'demo1'
conn_demo1 = psycopg2.connect(
    host="localhost",
    database="demo1",
    user="postgres",
    password="password"
)

cursor_demo1 = conn_demo1.cursor()

# Truy vấn dữ liệu từ tbl_user
cursor_demo.execute("SELECT id, fullname, phone, created_at FROM tbl_user")
users = cursor_demo.fetchall()

# # Lưu dữ liệu vào user_dim trong cơ sở dữ liệu 'demo1'
for user in users:
    cursor_demo1.execute("""
        INSERT INTO user_dim (id, fullname, phone, created_at)
        VALUES (%s, %s, %s, %s)
    """, (user[0], user[1], user[2], user[3]))

# # Truy vấn dữ liệu từ tbl_product
cursor_demo.execute("SELECT id, price, image, created_at FROM tbl_product")
products = cursor_demo.fetchall()

# # Lưu dữ liệu vào product_dim trong cơ sở dữ liệu 'demo1'
for product in products:
    cursor_demo1.execute("""
        INSERT INTO product_dim (id, price, image, created_at)
        VALUES (%s, %s, %s, %s)
    """, (product[0], product[1], product[2], product[3]))

# # Truy vấn dữ liệu từ tbl_order
cursor_demo.execute("select id, order_id, product_id, price, quantity, created_at FROM tbl_order_item")
orders = cursor_demo.fetchall()

# Lưu dữ liệu vào order_item_dim trong cơ sở dữ liệu 'demo1'
for order in orders:
    cursor_demo1.execute("""
        INSERT INTO order_item_dim (id, order_id, product_id, price, quantity, created_at)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (order[0], order[1], order[2], order[3], order[4], order[5]))

# Trường product_id và quantity có thể cần được điều chỉnh tùy theo cấu trúc dữ liệu của tbl_order và mối quan hệ với tbl_product

# Commit và đóng kết nối
conn_demo1.commit()
cursor_demo1.close()
conn_demo1.close()

In [29]:
import psycopg2
from psycopg2 import sql
from datetime import datetime

# Kết nối đến cơ sở dữ liệu 'demo'
conn_demo = psycopg2.connect(
    dbname="demo",
    user="postgres",
    password="password",
    host="localhost",
    port="5432"
)
cursor_demo = conn_demo.cursor()

# Kết nối đến cơ sở dữ liệu 'demo1'
conn_demo1 = psycopg2.connect(
    dbname="demo1",
    user="postgres",
    password="password",
    host="localhost",
    port="5432"
)
cursor_demo1 = conn_demo1.cursor()

# Thực hiện truy vấn SELECT dữ liệu từ bảng tbl_order
select_query = """
    SELECT id, created_at
    FROM tbl_order
"""
cursor_demo.execute(select_query)
orders = cursor_demo.fetchall()

# Xử lý từng dòng dữ liệu
for order in orders:
    order_id = order[0]
    created_at = order[1]
    
    # Tính toán các giá trị cần lưu vào bảng time_dim
    year = created_at.year
    quarter = (created_at.month - 1) // 3 + 1
    month = created_at.month

    # Chuyển đổi created_at thành chuỗi định dạng datetime
    created_at_str = created_at.strftime('%Y-%m-%d %H:%M:%S')

    # Create INSERT query for time_dim table
    insert_query = sql.SQL("""
        INSERT INTO time_dim (id, day, week, month, quarter, year, created_at)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """).format(
        sql.Identifier(order_id),  # Use order_id as identifier directly
        sql.Identifier(str(created_at.day)),  # Convert day to string and use as identifier
        sql.Identifier(str(0)),  # Convert week value to string and use as identifier
        sql.Identifier(str(month)),  # Convert month to string and use as identifier
        sql.Identifier(str(quarter)),  # Convert quarter to string and use as identifier
        sql.Identifier(str(year)),  # Convert year to string and use as identifier
        sql.Identifier(created_at_str),  # Use created_at as parameter directly
    )

    # Execute INSERT query
    cursor_demo1.execute(insert_query, (order_id, created_at.day, 0, month, quarter, year, created_at_str))


# Lưu các thay đổi vào database 'demo1'
conn_demo1.commit()


# Đóng kết nối
cursor_demo.close()
cursor_demo1.close()
conn_demo.close()
conn_demo1.close()



In [ ]:
import psycopg2

# Kết nối tới database nguồn
source_conn = psycopg2.connect(
    dbname="demo",
    user="postgres",
    password="password",
    host="localhost",
    port="5432"


source_cur = source_conn.cursor()

# Kết nối tới database đích
target_conn = psycopg2.connect(
    dbname="demo1",
    user="postgres",
    password="password",
    host="localhost",
    port="5432"


target_cur = target_conn.cursor()

# Truy vấn dữ liệu từ bảng nguồn
source_cur.execute("SELECT * FROM tbl_order")
rows = source_cur.fetchall()

# Duyệt qua từng dòng dữ liệu
for row in rows:
    # Trích xuất dữ liệu từ dòng hiện tại
    id, user_id, payment_type, discount, total_cost, fee_total, created_at = row
    # Chèn dữ liệu vào bảng đích
    target_cur.execute("INSERT INTO order_dim (id, user_id, payment_type, discount, total_cost, fee_total, created_at) "
                       "VALUES (%s, %s, %s, %s, %s, %s, %s)",
                       (id, user_id, payment_type, discount, total_cost, fee_total, created_at))

# Lưu các thay đổi vào database đích
target_conn.commit()

# Đóng kết nối
source_cur.close()
source_conn.close()
target_cur.close()
target_conn.close()